<a href="https://colab.research.google.com/github/andrewpkitchin/google-n-grams/blob/main/google_n_grams_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import requests, json, csv, time

In [13]:
# 1500-2019

list_of_dates = []

for i in range(0,520):
  list_of_dates.append(1500+i)

In [14]:
# takes a list of grams and date labels and creates a csv where each cell represent the google n-gram percentage for a gram and a year

def ngrams_to_csv(csv_name, list_of_ngrams, list_of_dates):
  with open(csv_name, 'w', newline='') as file:
    writer = csv.writer(file)

    list_of_dates.insert(0,'Word')

    # Write the headings to the csv.

    writer.writerow(list_of_dates)

    list_of_dates.pop(0)

    # smoothing = 0 means no smoothing, list of copora 

    for word in list_of_ngrams:
      url =f"https://books.google.com/ngrams/json?content={word}&year_start=1500&year_end=2019&corpus=26&smoothing=0&case_insensitive=case_insensitive"
      resp = requests.get(url)
      if resp.ok:
        results = json.loads(resp.content)

      listOfPercentages = [word] + results[0]['timeseries']

      # Writing the n-gram percentages to the csv.

      writer.writerow(listOfPercentages)

      time.sleep(0.5)

In [ ]:
list_of_ngrams = ["one", "Albert Einstein"]

In [ ]:
ngrams_to_csv("test1.csv", list_of_ngrams, list_of_dates)

In [15]:
# date_indexer(1950,2019) RETURNS [450, 520] 

def date_indexer(start_date, end_date):
  start = start_date-1500
  end = end_date-1499
  return [start, end]

In [25]:
# for the list results[0]['timeseries'] containing percentages for 1500-2019, average_over_dates(list_of_ngrams, start_date, end_date) returns average of 
# percentage in the date range 

def average_over_dates(list_of_ngrams, start_date, end_date):
  indexes = date_indexer(start_date, end_date)
  sliced_list = list_of_ngrams[indexes[0]:indexes[1]]
  average = sum(sliced_list)/len(sliced_list)

  return average 

In [17]:
def average_percentage_over_range(list_of_ngrams, start_date, end_date):
  
  holding_list = [] 

  for gram in list_of_ngrams:
        url =f"https://books.google.com/ngrams/json?content={gram}&year_start=1500&year_end=2019&corpus=26&smoothing=0&case_insensitive=case_insensitive"
        resp = requests.get(url)
        if resp.ok:
          results = json.loads(resp.content)

        holding_list.append(average_over_dates(results[0]['timeseries'],start_date, end_date))

        time.sleep(0.2)

  average = sum(holding_list)/len(holding_list)

  return average

In [27]:
# entities_and_queries = [[entity, [list of queries]], ... ] 

def ngrams_averages_to_csv(csv_name, entities_and_queries, start_date, end_date):

  with open(csv_name, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the headings to the csv.

    writer.writerow(['entity', 'average percentage', 'list of queries'])
    
    for entry in entities_and_queries:

      result = average_percentage_over_range(entry[1], start_date, end_date)

      writer.writerow([entry[0], result, entry[1]])

In [28]:
# test

entities_and_queries = [['one',['one two', 'hello everyone']]]

ngrams_averages_to_csv('test.csv', entities_and_queries, 1950, 2019)

In [ ]:
# version 2 

In [30]:
# entities_and_queries = [[entity, [perception term, query one, query two, ...], [perception term, query one, query two, ...] ], ... ] 

def ngrams_averages_to_csv(csv_name, entities_and_queries, start_date, end_date):

  with open(csv_name, 'w', newline='') as file:
    writer = csv.writer(file)

    # Write the headings to the csv.

    keyterms = []

    for entry in entities_and_queries:
      
      for item in entry[1:]:
        keyterms.append(item[0])

    writer.writerow(['entity'] + keyterms)

    holding_list = []

    for entry in entities_and_queries:

      for item in entry[1:]:
        result = average_percentage_over_range(item[1:], start_date, end_date)

        holding_list.append(result)

      writer.writerow([entry[0]]+holding_list)



In [31]:
# test

entities_and_queries = [['one',['one two', 'hello everyone'], ['three', 'hello there']]]

ngrams_averages_to_csv('test.csv', entities_and_queries, 1950, 2019)